In [61]:
import numpy as np
import pandas as pd
import cv2
import Augmentor
import mediapipe as mp
from os import listdir, mkdir, makedirs
from os.path import join, isfile
import pickle
import math

In [3]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5, max_num_hands=1)

In [63]:
srcPth = "../images/output/"
outPth = "../output/"
dirs = ["A", "B", "C", "D", "E", "F", "G" ,"H", "I", "K", "L", "M", "N", "O", "P", "Q", "R" ,"S" ,"T" ,"U" ,"V" ,"W" ,"X" ,"Y"]

for m in range(len(dirs)):
    try:
        makedirs(outPth + str(dirs[m]))
    except FileExistsError:
        continue

In [64]:
srcPth = "../images/output/"
outPth = "../output/"
dirs = ["A", "B", "C", "D", "E", "F", "G" ,"H", "I", "K", "L", "M", "N", "O", "P", "Q", "R" ,"S" ,"T" ,"U" ,"V" ,"W" ,"X" ,"Y"]
lmks = []
offset = 20
imgSize = 200


for i in range(len(dirs)):
    pth = srcPth + dirs[i]
    for j in range(len(listdir(pth))):

        imgAddr = pth + "/" + listdir(pth)[j]
        img = cv2.imread(imgAddr)
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        res = hands.process(imgRGB)
        lmk = []
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255

        if res.multi_hand_landmarks:
            for hand_landmarks in res.multi_hand_landmarks:
                #for landmark in hand_landmarks.landmark:
                hll = hand_landmarks.landmark
                w = (max(lmk.x for lmk in hll) - min(lmk.x for lmk in hll))*imgRGB.shape[1]
                h = (max(lmk.y for lmk in hll) - min(lmk.y for lmk in hll))*imgRGB.shape[0]
                x = (min(lmk.x for lmk in hll)*imgRGB.shape[1])
                y = (min(lmk.y for lmk in hll)*imgRGB.shape[0])

                imgCrop = img[int(y-offset):int(y+h+offset), int(x-offset):int(x+w+offset)]
        if imgCrop.size != 0:
            imgCropShape = imgCrop.shape
            aspectRatio = h / w
            if aspectRatio >+ 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                imgResizeShape = imgResize.shape
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap:wCal + wGap] = imgResize
            else:
                k = imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                imgResizeShape = imgResize.shape
                hGap = math.ceil((imgSize - hCal) / 2)
                imgWhite[hGap:hCal + hGap, :] = imgResize
                
                
            filePth = outPth + str(dirs[i]) + "/" + str(j) + ".jpg"
            cv2.imwrite(filePth, imgWhite)